# Simulate the striatum example network

This example continues ```striatum_example.ipynb``` and shows how to setup input and run the simulation.

## Setup the input

Here we use the file ```input_config/striatum-test-input.json```.

The input can be generated in serial or in parallel. To generate it in parallel, pass ```rc``` parameter as in the previous examples.

In [1]:
import os
from snudda.input import SnuddaInput

network_path = os.path.join("networks", "striatum_example")

si = SnuddaInput(network_path=network_path, 
                 input_config_file=os.path.join("input_config","striatum-test-input.json"),
                 verbose=False)
si.generate()

Reading SNUDDA_DATA=../../../BasalGangliaData/data/ from networks/striatum_example/network-config.json
Loading 20 neurons with 344 synapses and 0 gap junctions
Writing spikes to networks/striatum_example/input-spikes.hdf5


## Compile the  NEURON mod files
If you run this on Windows you need to exchange / for \ in the path below. Also there are two versions below, the first one is if you dont have basal ganglia data, and the one after is if you have basal ganglia data.

In [2]:
# !nrnivmodl ../../snudda/data/neurons/mechanisms

/home/hjorth/HBP/Snudda/examples/notebooks
Mod files: "../../snudda/data/neurons/mechanisms/../../snudda/data/neurons/mechanisms/bk_ch.mod" "../../snudda/data/neurons/mechanisms/../../snudda/data/neurons/mechanisms/bk_fs.mod" "../../snudda/data/neurons/mechanisms/../../snudda/data/neurons/mechanisms/bk_ms.mod" "../../snudda/data/neurons/mechanisms/../../snudda/data/neurons/mechanisms/ca_ch.mod" "../../snudda/data/neurons/mechanisms/../../snudda/data/neurons/mechanisms/cadyn_fs.mod" "../../snudda/data/neurons/mechanisms/../../snudda/data/neurons/mechanisms/cadyn_ms.mod" "../../snudda/data/neurons/mechanisms/../../snudda/data/neurons/mechanisms/cal12_ms.mod" "../../snudda/data/neurons/mechanisms/../../snudda/data/neurons/mechanisms/cal13_ms.mod" "../../snudda/data/neurons/mechanisms/../../snudda/data/neurons/mechanisms/cal_ch.mod" "../../snudda/data/neurons/mechanisms/../../snudda/data/neurons/mechanisms/caldyn_ms.mod" "../../snudda/data/neurons/mechanisms/../../snudda/data/neurons/mecha

In [ ]:
!nrnivmodl ../../../BasalGangliaData/data/neurons/mechanisms

## Run the neuron simulation

Here we run the network for 3.5 seconds.

In [ ]:
!mpiexec -n 4 snudda simulate networks/striatum_example --time 3.5

See ```striatum_example_plot.ipynb``` for information on plotting a spike raster.